In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

visit_origin = os.environ.get("SCHEDVIEW_VISIT_ORIGIN", "baseline")
day_obs = int(
    os.environ.get(
        "SCHEDVIEW_DAY_OBS",
        (
            (datetime.date.today() - datetime.timedelta(days=1)).isoformat()
            if visit_origin in ("lsstcam", "latiss", "lsstcomcam")
            else "2026-12-21"
        ),
    ).replace("-", "")
)


In [ ]:
import os
import sys

#sched_source = 'env'
sched_source = 'shared'
#sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.11.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev7+g8c79ae0")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use whatever is in the kernel python environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
import datetime
import os
import sqlite3
import sys
from collections import defaultdict
from pathlib import Path

import astropy
import astropy.units as u
import bokeh
import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting
import bokeh.transform
import cartopy
import colorcet
import healpy
import healpy as hp
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from erfa import ErfaWarning
from astropy.coordinates import SkyCoord, get_body
from astropy.time import Time
from astropy.visualization import ZScaleInterval
from IPython.display import HTML, display, Markdown, HTML
from lsst.resources import ResourcePath

In [ ]:
usdf_sim_data_dir = "/sdf/data/rubin/shared/rubin_sim_data"
if os.path.exists(usdf_sim_data_dir):
    os.environ["RUBIN_SIM_DATA_DIR"] = "/sdf/data/rubin/shared/rubin_sim_data"

In [ ]:
import schedview.collect.consdb
import schedview.collect.nightreport
import schedview.collect.timeline
import rubin_scheduler
import rubin_scheduler.site_models
import rubin_scheduler.utils
import schedview.compute.astro
import schedview.compute.visits
import schedview.compute.nightreport
import schedview.plot.survey_skyproj
import schedview.plot.visitmap
import schedview.plot.nightreport
import schedview.collect.visits
import rubin_sim.sim_archive
import uranography
from rubin_sim.data import get_baseline
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from schedview.compute.camera import LsstCameraFootprintPerimeter
from uranography.api import Planisphere, make_zscale_linear_cmap
from schedview import DayObs
from rubin_scheduler.utils.consdb import KNOWN_INSTRUMENTS

In [ ]:
if not Path('/sdf/group/rubin/web_data/sim-data').exists():
    schedview.collect.visits.OPSIMDB_TEMPLATE = str(
        Path.home().joinpath("Data/sim-data/sims_featureScheduler_runs{sim_version}/baseline/baseline_v{sim_version}_10yrs.db")
    )

In [ ]:
# Validate the input
import re

assert visit_origin in KNOWN_INSTRUMENTS or visit_origin == 'baseline' or re.match(r"^\d+\.\d+$", visit_origin) is not None

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

In [ ]:
bokeh.io.output_notebook(hide_banner=True)

In [ ]:
%matplotlib inline

In [ ]:
ORIGIN_TELESCOPE = defaultdict(
    np.array(['Simonyi']).item,
    {'latiss': 'AuxTel'}
)

In [ ]:
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"
telescope = ORIGIN_TELESCOPE[visit_origin]
from_opsim = visit_origin not in KNOWN_INSTRUMENTS
use_matplotlib = True
science_programs_to_include = ['BLOCK-365']
telescope = "AuxTel" if visit_origin.lower()=="latiss" else "Simonyi"

In [ ]:
if from_opsim:
    display(Markdown(f"# Scheduler-oriented night summary of {day_obs} from baseline {visit_origin}"))
else:
    display(Markdown(f"# Scheduler-oriented night summary of {visit_origin} on {day_obs}"))

This report is a summary of a night of observing designed to understand scheduler behavior during the night, identify scheduler problems, and otherwise monitor scheduler performance.

In [ ]:
day_obs = DayObs.from_date(day_obs)
observatory.mjd = day_obs.mjd + 1 - observatory.location.lon.deg/360 ;# The approximate middle of the night

In [ ]:
visits = schedview.collect.visits.read_visits(day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS)

In [ ]:
if not from_opsim:
    visits = visits.loc[visits['science_program'].isin(science_programs_to_include), :]

In [ ]:
night_events = schedview.compute.astro.night_events(day_obs.date)

In [ ]:
# Refine the mjd of the middle of the night, unlikely to be important.
observatory.mjd = night_events.loc["night_middle", "MJD"]

In [ ]:
body_positions = schedview.compute.astro.compute_sun_moon_positions(observatory)

## Visit map

In [ ]:
if len(visits):
    visits = schedview.compute.visits.add_coords_tuple(visits)
    vmap, vmap_data = schedview.plot.visitmap.create_visit_skymaps(
        visits=visits,
        night_date=day_obs.date,
        timezone=timezone,
        observatory=observatory,
    )
    bokeh.io.show(vmap)
else:
    print("No visits")

The above plots show the visits collected during the night in two different representations, modeled after physical observing tools.

 - The "Armillary sphere" shows the sphere in orthographic projection, with the center point of the projection controlled by the "center alt" and "center az" sliders beneath the plot. A static orthogrophic projection is not an equal-area projection, but playing with the sliders is a helpful way to inform a human's spatial reasoning in three dimensions.
 - The "Planisphere" shows the sky in [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole, with R.A. increasing counterclockwise. The projection used is equal area, but highly distorted near the north celestial pole (outside the LSST footprint). This is a particularly helpful representation for planning observing, because changes in time in relevant features are simple rotations, without alterations in distortion, and there are no discontinuities anywhere in the footprint at any time of year.

Both plots show the footprints of camera pointing taken up to the time set by the MJD slider, with the most recent three pointings outlined in cyan. The fill colors are set according to the [descolors palette](https://github.com/DarkEnergySurvey/descolors):

 - <span style='background-color:#56b4e9'>&nbsp;&nbsp;&nbsp;</span><span style='color:#56b4e9'> blue</span>: u band
 - <span style='background-color:#008060'>&nbsp;&nbsp;&nbsp;</span><span style='color:#008060'> green</span>: g band
 - <span style='background-color:#ff4000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#ff4000'> red</span>: r band
 - <span style='background-color:#850000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#850000'> brown/crimson</span>: i band
 - <span style='background-color:#6600cc'>&nbsp;&nbsp;&nbsp;</span><span style='color:#6600cc'> purple</span>: z band
 - <span style='background-color:#000000'>&nbsp;&nbsp;&nbsp;</span><span style='color:#000000'> black</span>: y band

Both plots have the following additional annotations:
 - The gray background shows the planned final depth of the LSST survey.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
   - The sun moves along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) such that it makes a full revolution in one year.
   - The moon moves roughly (within 5.14°) along the ecliptic in the direction of increasing R.A. (counter-clockwise in the planisphere figure) , completing a full revolution in one [sidereal month](https://en.wikipedia.org/wiki/Lunar_month#Sidereal_month) (a bit over 27 days), about 14° per day.
 - The blue line (oval) shows the plane of the Milky Way.
 - The black line shows the horizon at the time set by the MJD slider.
 - The red line shows a zenith distince of 70° (airmass=2.9) at the time set by the MJD slider.

## Value timelines

Select a value to plot using the dropdown above the plot on the right.

In [ ]:
if len(visits):
    fig = schedview.plot.plot_visit_param_vs_time(
        visits, 
        'airmass',
        plot=bokeh.plotting.figure(height=512, width=1024),
        show_column_selector=True)
    bokeh.io.show(fig)
else:
    print("No visits")


## Survey Progress

### Map numbers of visits accumulated so far

In [ ]:
conditions = observatory.return_conditions()

In [ ]:
previous_day_obs = DayObs.from_date(day_obs.mjd-1, int_format='mjd')
previous_visits = schedview.collect.visits.read_visits(previous_day_obs, visit_origin, stackers = schedview.collect.visits.NIGHT_STACKERS, num_nights=10000)
if not from_opsim:
    previous_visits = previous_visits.loc[previous_visits['science_program'].isin(science_programs_to_include), :]

In [ ]:
if len(visits) and len(previous_visits) \
        and not np.all(np.isnan(visits['fiveSigmaDepth'])) \
        and not np.all(np.isnan(previous_visits['fiveSigmaDepth'])):
    fig = schedview.plot.survey.create_metric_visit_map_grid(
        maf.CountMetric(col='fiveSigmaDepth', metric_name="Numbers of visits"),
        previous_visits.loc[np.isfinite(previous_visits['fiveSigmaDepth']), :],
        visits.loc[np.isfinite(visits['fiveSigmaDepth']), :],
        observatory,
        nside=32,
        use_matplotlib=use_matplotlib
    )

Each subplot represents the progress in the survey in a different filter, presented in a [Lambert Azimuthal Equal Area Projection](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection), centered at the south celestial pole.

Annotations are similar to those of the "Visit Map" above:

 - Black outlines show the camera footprint of each exposure taken on this night.
 - The blue backdrop represents the accumulated effective exposure time, t<sub>eff</sub> (as defined above) at the start of the night.
 - The orange disk shows the coordinates of the moon.
 - The yellow disk shows the coordinates of the sun.
 - The green line (oval) shows the ecliptic.
 - The blue line (oval) shows the plane of the Milky Way.
 - Broken red ovals show a zenith distince of 70° (airmass=2.9) at morning and evening 12° twilight.
   - The dashed portions of each red oval show the extent of what is observable at all during the night.
   - As time progress through the night, the ZD circle maintains its shape, moving counter-clockwise around the pole over the course of the night.

### Map the most recent visit so far

In [ ]:
if len(previous_visits):
    fig = schedview.plot.survey.create_metric_visit_map_grid(
        maf.AgeMetric(night_events.loc['sunset', 'MJD']),
        previous_visits,
        visits,
        observatory,
        nside=32,
        use_matplotlib=use_matplotlib,
        vmin=0,
        vmax=10,
        cmap=colorcet.cm.blues_r
    )

Each subplot represents the time since the most recent visits to each point in the sky, by filter, presented in a Lambert Azimuthal Equal Area Projection, centered at the south celestial pole.

Parts of the sky visited most recently are dark blue, growing fainter and grayer for areas visited less recently. Regions not visited in the last 10 days are not colored.

Other features and annotations are the same as those of the depth map, above.

## DDF Cadence

In [ ]:
time_window_duration=90

In [ ]:
try:
    ddf_visits = schedview.collect.visits.read_ddf_visits(day_obs, visit_origin, num_nights=time_window_duration)
    if not from_opsim:
        ddf_visits = ddf_visits.loc[ddf_visits['science_program'].isin(science_programs_to_include), :]
except:
    ddf_visits = []

The y-axis (height of the vertical bars) represents the accumulated effective exposure time, teff (as defined above) accumulated over all exposures on the field for the night, colored by filter.

In [ ]:
list(visits.columns)